# LangChain Expression Language (LCEL)

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from helper import show_response

## Simple Chain

In [2]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the life of the "bear"-ty!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [5]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [6]:
vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [7]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [11]:
retriever.invoke("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain.schema.runnable import RunnableMap

chain = (
    RunnableMap(
        {
            "context": lambda x: retriever.invoke(x["question"]),
            "question": lambda x: x["question"],
        }
    )
    | prompt
    | model
    | output_parser
)

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [15]:
inputs = RunnableMap(
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],
    }
)

In [16]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [17]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for",
                },
            },
            "required": ["airport_code"],
        },
    }
]

In [18]:
prompt = ChatPromptTemplate.from_messages([("human", "{input}")])
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [19]:
runnable = prompt | model

In [22]:
show_response(runnable.invoke({"input": "what is the weather in sf"}))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"airport_code\":\"SFO\"}",
      "name": "weather_search"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 16,
      "prompt_tokens": 64,
      "total_tokens": 80,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxHelby6qYjsfZqFXnK8vRl65vFOs",
    "service_tier": "default",
    "finish_reason": "function_call",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--9d90808e-90b9-42cf-8079-55a03c050e92-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "inp

In [23]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for",
                },
            },
            "required": ["airport_code"],
        },
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for",
                },
            },
            "required": ["team_name"],
        },
    },
]

In [24]:
model = model.bind(functions=functions)

In [25]:
runnable = prompt | model

In [27]:
show_response(runnable.invoke({"input": "how did the patriots do yesterday?"}))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"team_name\":\"patriots\"}",
      "name": "sports_search"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 18,
      "prompt_tokens": 99,
      "total_tokens": 117,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxHfFYPn4c7lq09k0HxHoRFQV15Nv",
    "service_tier": "default",
    "finish_reason": "function_call",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--c8635435-ce71-4354-b0fc-2030da92dd86-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "i

## Fallbacks

In [28]:
import json

# old model
from langchain.llms import OpenAI

In [29]:
simple_model = OpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo-instruct")
simple_chain = simple_model | json.loads

/var/folders/xf/7hwvtm5j57s6dtr8_dq1g97m0000gn/T/ipykernel_19349/920975571.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo-instruct")


In [30]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [32]:
print(simple_model.invoke(challenge))



{
    "title": "Autumn Leaves",
    "author": "Emily Dickinson",
    "first_line": "The leaves are falling, one by one"
}

{
    "title": "The Ocean's Song",
    "author": "Pablo Neruda",
    "first_line": "I hear the ocean's song, a symphony of waves"
}

{
    "title": "A Winter's Night",
    "author": "Robert Frost",
    "first_line": "The snow falls softly, covering the ground"
}


<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [33]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [34]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [35]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [36]:
final_chain = simple_chain.with_fallbacks([chain])

In [37]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface

In [38]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [39]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food? Because they can't catch it!"

In [40]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\nBecause they prefer bear feet!",
 'Why are frogs so happy? Because they eat whatever bugs them! 🐸']

In [41]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 break
 up
 with
 his
 girlfriend
?
 


Because
 he
 couldn
't
 bear
 the
 relationship
 


Because
 he
 couldn
't
 bear
 the
 relationship
 anymore
!

 anymore
!



In [42]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear become a referee? Because he was great at bear-y-ing the whistle!'